<a href="https://colab.research.google.com/github/swiminthewind/colab/blob/main/240113-%E5%BE%97%E5%88%B03class%E6%A0%B7%E6%9C%AC%E7%9A%842022NDVI%E6%97%B6%E9%97%B4%E5%BA%8F%E5%88%97.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import ee
ee.Authenticate()
ee.Initialize(project='ee-grn')

In [7]:
import os
import pandas as pd
import math

In [8]:
roi = ee.FeatureCollection("projects/ee-grn/assets/cd_plain_cd")
csPlus= ee.ImageCollection("GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED")
s2sr = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
point = ee.FeatureCollection("projects/ee-grn/assets/sample_3class")

In [9]:
timeField = 'system:time_start'
QA_BAND = 'cs'
CLEAR_THRESHOLD = 0.6

def cloud_remove(img):
    return img.updateMask(img.select(QA_BAND).gte(CLEAR_THRESHOLD)).select('.*').copyProperties(img, ['system:time_start'])

composite = s2sr.filterBounds(roi).filterDate('2022-01-01','2023-12-08').linkCollection(csPlus,[QA_BAND]).map(cloud_remove)

def s2addVariables(image):
  date = ee.Date(image.get(timeField))
  years = date.difference(ee.Date('1970-01-01'), 'year')
  return image.addBands(image.normalizedDifference(['B8', 'B4']).rename('NDVI')).addBands(ee.Image(years).rename('t')).float().addBands(ee.Image.constant(1))

filteredSentinel = composite.map(s2addVariables)
print(filteredSentinel.size().getInfo())

1279


In [10]:
# 二阶谐波
dataset = filteredSentinel.select(['NDVI','t','constant'])
independents = ee.List(['constant', 't'])
dependent = ee.String('NDVI')

harmonicIndependents = ee.List(['constant', 't', 'cos1', 'sin1', 'cos2', 'sin2'])

def add_sincos(image):
  timeRadians1 = image.select('t').multiply(2 * math.pi)
  timeRadians2 = image.select('t').multiply(4 * math.pi)
  return image.addBands(timeRadians1.cos().rename('cos1')).addBands(timeRadians1.sin().rename('sin1')).addBands(timeRadians2.cos().rename('cos2')).addBands(timeRadians2.sin().rename('sin2'))

harmonicSentinel = dataset.map(add_sincos)

harmonicTrend = harmonicSentinel.select(harmonicIndependents.add(dependent)).reduce(ee.Reducer.linearRegression(numX=harmonicIndependents.length(),numY=1))

harmonicTrendCoefficients = harmonicTrend.select('coefficients').arrayProject([0]).arrayFlatten([harmonicIndependents])

def fit(image):
  return image.addBands(image.select(harmonicIndependents).multiply(harmonicTrendCoefficients).reduce('sum').rename('fitted'))

fittedHarmonic = harmonicSentinel.map(fit)

print(fittedHarmonic.size().getInfo())

1279


In [11]:
# 5天序列NDVI
interval = 5
increment = 'day'
start = '2022-05-01'
startDate = ee.Date(start)
secondDate = startDate.advance(interval, increment).millis()
increase = secondDate.subtract(startDate.millis())
list5d = ee.List.sequence(startDate.millis(), ee.Date('2022-09-01').millis(), increase)

def add_time(date):
  return ee.Image(0).mask(0).rename('x').set('system:time_start',ee.Date(date).millis())

dailyMean =  ee.ImageCollection.fromImages(list5d.map(add_time))

def add_allbands(image):
  date = ee.Date(image.get(timeField))
  years = date.difference(ee.Date('1970-01-01'), 'year')
  timeRadians1 = ee.Image(years).multiply(2 * math.pi)
  timeRadians2 = ee.Image(years).multiply(4 * math.pi)
  return image.addBands(ee.Image(years).rename('t')).float().addBands(ee.Image.constant(1)).addBands(timeRadians1.cos().rename('cos1')).addBands(timeRadians1.sin().rename('sin1')).addBands(timeRadians2.cos().rename('cos2')).addBands(timeRadians2.sin().rename('sin2'))

s2makeup = dailyMean.map(add_allbands)

def add_fittedNDVI(image):
  return image.addBands(image.select(harmonicIndependents).multiply(harmonicTrendCoefficients).reduce('sum').rename('NDVI'))

fitteds2 = s2makeup.map(add_fittedNDVI)

HANTS = fitteds2.select('NDVI')

In [12]:
# NDVI时间序列合成
image_agrgt = HANTS.toBands()
print(image_agrgt.bandNames().getInfo())

['0_NDVI', '1_NDVI', '2_NDVI', '3_NDVI', '4_NDVI', '5_NDVI', '6_NDVI', '7_NDVI', '8_NDVI', '9_NDVI', '10_NDVI', '11_NDVI', '12_NDVI', '13_NDVI', '14_NDVI', '15_NDVI', '16_NDVI', '17_NDVI', '18_NDVI', '19_NDVI', '20_NDVI', '21_NDVI', '22_NDVI', '23_NDVI', '24_NDVI']


In [13]:
image_agrgt_clip = image_agrgt.clip(roi)

In [23]:
samples_datesets=image_agrgt_clip.sampleRegions(collection=point,properties=['id','landcover'],scale=10,tileScale=8)

In [24]:
cols = ['id', 'landcover', '0_NDVI','1_NDVI','2_NDVI','3_NDVI','4_NDVI','5_NDVI','6_NDVI','7_NDVI','8_NDVI','9_NDVI','10_NDVI','11_NDVI','12_NDVI','13_NDVI','14_NDVI','15_NDVI','16_NDVI','17_NDVI','18_NDVI','19_NDVI','20_NDVI','21_NDVI','22_NDVI','23_NDVI','24_NDVI']
inds = range(samples_datesets.size().getInfo())

In [25]:
df = pd.DataFrame(index=inds,columns=cols)
df.head()

,id,landcover,0_NDVI,1_NDVI,2_NDVI,3_NDVI,4_NDVI,5_NDVI,6_NDVI,7_NDVI,...,15_NDVI,16_NDVI,17_NDVI,18_NDVI,19_NDVI,20_NDVI,21_NDVI,22_NDVI,23_NDVI,24_NDVI
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
data_value = samples_datesets.reduceColumns(
    reducer = ee.Reducer.toList().repeat(27),
    selectors = cols
)
df = pd.DataFrame(data_value.getInfo()['list'],cols)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,237,238,239,240,241,242,243,244,245,246
id,52.000000,54.000000,55.000000,56.000000,57.000000,58.000000,59.000000,61.000000,62.000000,63.000000,...,42.000000,43.000000,44.000000,45.000000,46.000000,47.000000,48.000000,49.000000,50.000000,51.000000
landcover,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
0_NDVI,0.388970,0.567794,0.551880,0.470970,0.427335,0.336329,0.421983,0.614099,0.679539,0.403990,...,0.475452,0.356384,0.340565,0.484746,0.361191,0.402057,0.215890,0.291169,0.388050,0.315157
1_NDVI,0.379486,0.566675,0.537553,0.456529,0.413515,0.331527,0.423733,0.597763,0.663470,0.379317,...,0.492279,0.338753,0.349724,0.465339,0.343744,0.398884,0.252265,0.277780,0.368129,0.312887
2_NDVI,0.376208,0.570766,0.529307,0.448670,0.403692,0.330432,0.429280,0.585846,0.649975,0.362212,...,0.509480,0.324709,0.359744,0.449770,0.329769,0.398493,0.291274,0.268138,0.353417,0.314367


In [27]:
df.to_csv('sample_NDVI_2022_247.csv',encoding='utf-8')